In [1]:
library(data.table)
library(zoo)
library(forecast)
library(ggplot2)


Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: timeDate
This is forecast 7.1 



In [2]:
test = fread("test.csv")
train = fread("train.csv")
store = fread("store.csv")

In [3]:
str(train)

Classes ‘data.table’ and 'data.frame':	1017209 obs. of  9 variables:
 $ Store        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ DayOfWeek    : int  5 5 5 5 5 5 5 5 5 5 ...
 $ Date         : chr  "2015-07-31" "2015-07-31" "2015-07-31" "2015-07-31" ...
 $ Sales        : int  5263 6064 8314 13995 4822 5651 15344 8492 8565 7185 ...
 $ Customers    : int  555 625 821 1498 559 589 1414 833 687 681 ...
 $ Open         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Promo        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ StateHoliday : chr  "0" "0" "0" "0" ...
 $ SchoolHoliday: chr  "1" "1" "1" "1" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [4]:
str(test)

Classes ‘data.table’ and 'data.frame':	41088 obs. of  8 variables:
 $ Id           : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Store        : int  1 3 7 8 9 10 11 12 13 14 ...
 $ DayOfWeek    : int  4 4 4 4 4 4 4 4 4 4 ...
 $ Date         : chr  "2015-09-17" "2015-09-17" "2015-09-17" "2015-09-17" ...
 $ Open         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Promo        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ StateHoliday : chr  "0" "0" "0" "0" ...
 $ SchoolHoliday: chr  "0" "0" "0" "0" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [5]:
str(store)

Classes ‘data.table’ and 'data.frame':	1115 obs. of  10 variables:
 $ Store                    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ StoreType                : chr  "c" "a" "a" "c" ...
 $ Assortment               : chr  "a" "a" "a" "c" ...
 $ CompetitionDistance      : int  1270 570 14130 620 29910 310 24000 7520 2030 3160 ...
 $ CompetitionOpenSinceMonth: int  9 11 12 9 4 12 4 10 8 9 ...
 $ CompetitionOpenSinceYear : int  2008 2007 2006 2009 2015 2013 2013 2014 2000 2009 ...
 $ Promo2                   : int  0 1 1 0 0 0 0 0 0 0 ...
 $ Promo2SinceWeek          : int  NA 13 14 NA NA NA NA NA NA NA ...
 $ Promo2SinceYear          : int  NA 2010 2011 NA NA NA NA NA NA NA ...
 $ PromoInterval            : chr  "" "Jan,Apr,Jul,Oct" "Jan,Apr,Jul,Oct" "" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [6]:
train[, Date := as.Date(Date)]
test[, Date := as.Date(Date)]
store

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1,5,2015-07-31,5263,555,1,1,0,1
2,5,2015-07-31,6064,625,1,1,0,1
3,5,2015-07-31,8314,821,1,1,0,1
4,5,2015-07-31,13995,1498,1,1,0,1
5,5,2015-07-31,4822,559,1,1,0,1
6,5,2015-07-31,5651,589,1,1,0,1
7,5,2015-07-31,15344,1414,1,1,0,1
8,5,2015-07-31,8492,833,1,1,0,1
9,5,2015-07-31,8565,687,1,1,0,1
10,5,2015-07-31,7185,681,1,1,0,1


Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
1,1,4,2015-09-17,1,1,0,0
2,3,4,2015-09-17,1,1,0,0
3,7,4,2015-09-17,1,1,0,0
4,8,4,2015-09-17,1,1,0,0
5,9,4,2015-09-17,1,1,0,0
6,10,4,2015-09-17,1,1,0,0
7,11,4,2015-09-17,1,1,0,0
8,12,4,2015-09-17,1,1,0,0
9,13,4,2015-09-17,1,1,0,0
10,14,4,2015-09-17,1,1,0,0


Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1,c,a,1270,9,2008,0,NA,NA,
2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
4,c,c,620,9,2009,0,NA,NA,
5,a,a,29910,4,2015,0,NA,NA,
6,a,a,310,12,2013,0,NA,NA,
7,a,c,24000,4,2013,0,NA,NA,
8,a,a,7520,10,2014,0,NA,NA,
9,a,c,2030,8,2000,0,NA,NA,
10,a,a,3160,9,2009,0,NA,NA,


In [7]:
train <- train[order(Date)]
test <- test[order(Date)]
summary(train)

     Store          DayOfWeek          Date                Sales      
 Min.   :   1.0   Min.   :1.000   Min.   :2013-01-01   Min.   :    0  
 1st Qu.: 280.0   1st Qu.:2.000   1st Qu.:2013-08-17   1st Qu.: 3727  
 Median : 558.0   Median :4.000   Median :2014-04-02   Median : 5744  
 Mean   : 558.4   Mean   :3.998   Mean   :2014-04-11   Mean   : 5774  
 3rd Qu.: 838.0   3rd Qu.:6.000   3rd Qu.:2014-12-12   3rd Qu.: 7856  
 Max.   :1115.0   Max.   :7.000   Max.   :2015-07-31   Max.   :41551  
   Customers           Open            Promo        StateHoliday      
 Min.   :   0.0   Min.   :0.0000   Min.   :0.0000   Length:1017209    
 1st Qu.: 405.0   1st Qu.:1.0000   1st Qu.:0.0000   Class :character  
 Median : 609.0   Median :1.0000   Median :0.0000   Mode  :character  
 Mean   : 633.1   Mean   :0.8301   Mean   :0.3815                     
 3rd Qu.: 837.0   3rd Qu.:1.0000   3rd Qu.:1.0000                     
 Max.   :7388.0   Max.   :1.0000   Max.   :1.0000                     
 Schoo

In [8]:
str(store)

Classes ‘data.table’ and 'data.frame':	1115 obs. of  10 variables:
 $ Store                    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ StoreType                : chr  "c" "a" "a" "c" ...
 $ Assortment               : chr  "a" "a" "a" "c" ...
 $ CompetitionDistance      : int  1270 570 14130 620 29910 310 24000 7520 2030 3160 ...
 $ CompetitionOpenSinceMonth: int  9 11 12 9 4 12 4 10 8 9 ...
 $ CompetitionOpenSinceYear : int  2008 2007 2006 2009 2015 2013 2013 2014 2000 2009 ...
 $ Promo2                   : int  0 1 1 0 0 0 0 0 0 0 ...
 $ Promo2SinceWeek          : int  NA 13 14 NA NA NA NA NA NA NA ...
 $ Promo2SinceYear          : int  NA 2010 2011 NA NA NA NA NA NA NA ...
 $ PromoInterval            : chr  "" "Jan,Apr,Jul,Oct" "Jan,Apr,Jul,Oct" "" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [9]:
total = merge(train, store, by='Store')

In [10]:
str(total)

Classes ‘data.table’ and 'data.frame':	1017209 obs. of  18 variables:
 $ Store                    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ DayOfWeek                : int  2 3 4 5 6 7 1 2 3 4 ...
 $ Date                     : Date, format: "2013-01-01" "2013-01-02" ...
 $ Sales                    : int  0 5530 4327 4486 4997 0 7176 5580 5471 4892 ...
 $ Customers                : int  0 668 578 619 635 0 785 654 626 615 ...
 $ Open                     : int  0 1 1 1 1 0 1 1 1 1 ...
 $ Promo                    : int  0 0 0 0 0 0 1 1 1 1 ...
 $ StateHoliday             : chr  "a" "0" "0" "0" ...
 $ SchoolHoliday            : chr  "1" "1" "1" "1" ...
 $ StoreType                : chr  "c" "c" "c" "c" ...
 $ Assortment               : chr  "a" "a" "a" "a" ...
 $ CompetitionDistance      : int  1270 1270 1270 1270 1270 1270 1270 1270 1270 1270 ...
 $ CompetitionOpenSinceMonth: int  9 9 9 9 9 9 9 9 9 9 ...
 $ CompetitionOpenSinceYear : int  2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 ...
 $ Pr

In [14]:
promoSales = glm(Promo~Sales, data=total[total$Sales < 3727])

In [15]:
summary(promoSales)


Call:
glm(formula = Promo ~ Sales, data = total[total$Sales < 3727])

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.10517  -0.08886  -0.06233  -0.06233   0.93767  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 6.233e-02  6.206e-04  100.43   <2e-16 ***
Sales       1.150e-05  3.584e-07   32.08   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.06772652)

    Null deviance: 17287  on 254218  degrees of freedom
Residual deviance: 17217  on 254217  degrees of freedom
AIC: 37018

Number of Fisher Scoring iterations: 2


In [13]:
summary(total)

     Store          DayOfWeek          Date                Sales      
 Min.   :   1.0   Min.   :1.000   Min.   :2013-01-01   Min.   :    0  
 1st Qu.: 280.0   1st Qu.:2.000   1st Qu.:2013-08-17   1st Qu.: 3727  
 Median : 558.0   Median :4.000   Median :2014-04-02   Median : 5744  
 Mean   : 558.4   Mean   :3.998   Mean   :2014-04-11   Mean   : 5774  
 3rd Qu.: 838.0   3rd Qu.:6.000   3rd Qu.:2014-12-12   3rd Qu.: 7856  
 Max.   :1115.0   Max.   :7.000   Max.   :2015-07-31   Max.   :41551  
                                                                      
   Customers           Open            Promo        StateHoliday      
 Min.   :   0.0   Min.   :0.0000   Min.   :0.0000   Length:1017209    
 1st Qu.: 405.0   1st Qu.:1.0000   1st Qu.:0.0000   Class :character  
 Median : 609.0   Median :1.0000   Median :0.0000   Mode  :character  
 Mean   : 633.1   Mean   :0.8301   Mean   :0.3815                     
 3rd Qu.: 837.0   3rd Qu.:1.0000   3rd Qu.:1.0000                     
 Max. 